In [1]:
!pip install torchtext==0.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 1.9 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.datasets import IMDB
from torchtext.data import Field, LabelField, BucketIterator

In [3]:
# Set device to use GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Define hyperparameters
batch_size = 64
num_epochs = 50
learning_rate = 0.001
hidden_dim1 = 500
hidden_dim2 = 300

In [5]:
# Define data transforms to preprocess input
TEXT = Field(lower=True, batch_first=True)
LABEL = LabelField(dtype=torch.float)

In [6]:
train_data, test_data = IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:04<00:00, 19.0MB/s]


In [7]:
TEXT.build_vocab(train_data, max_size=5000)
LABEL.build_vocab(train_data)

In [8]:
train_loader, test_loader = BucketIterator.splits(
    (train_data, test_data), batch_size=batch_size, device=device)

In [9]:
# Define neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.embedding = nn.Embedding(len(TEXT.vocab), 100)
        self.fc1 = nn.Linear(100, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, 1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x, dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        x = torch.sigmoid(x)
        return x.squeeze()

In [10]:
# Create neural network and move it to device
net = Net().to(device)

In [11]:
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [12]:
# Train neural network
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        inputs, labels = batch.text.to(device), batch.label.to(device)

        # Forward pass
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, len(train_loader), running_loss / 100))
            running_loss = 0.0

Epoch [1/50], Step [100/391], Loss: 0.6959
Epoch [1/50], Step [200/391], Loss: 0.6932
Epoch [1/50], Step [300/391], Loss: 0.6923
Epoch [2/50], Step [100/391], Loss: 0.6794
Epoch [2/50], Step [200/391], Loss: 0.6543
Epoch [2/50], Step [300/391], Loss: 0.5957
Epoch [3/50], Step [100/391], Loss: 0.4679
Epoch [3/50], Step [200/391], Loss: 0.4252
Epoch [3/50], Step [300/391], Loss: 0.4095
Epoch [4/50], Step [100/391], Loss: 0.3752
Epoch [4/50], Step [200/391], Loss: 0.3579
Epoch [4/50], Step [300/391], Loss: 0.3624
Epoch [5/50], Step [100/391], Loss: 0.3171
Epoch [5/50], Step [200/391], Loss: 0.3506
Epoch [5/50], Step [300/391], Loss: 0.3318
Epoch [6/50], Step [100/391], Loss: 0.3399
Epoch [6/50], Step [200/391], Loss: 0.3218
Epoch [6/50], Step [300/391], Loss: 0.3002
Epoch [7/50], Step [100/391], Loss: 0.2991
Epoch [7/50], Step [200/391], Loss: 0.2942
Epoch [7/50], Step [300/391], Loss: 0.2863
Epoch [8/50], Step [100/391], Loss: 0.2964
Epoch [8/50], Step [200/391], Loss: 0.3007
Epoch [8/50

In [13]:
# Define file path to save the trained model
model_path = 'two_layer_FFNN_model.pt'

# Save trained model
torch.save(net.state_dict(), model_path)

In [14]:
# Test neural network
with torch.no_grad():
    correct = 0
    total = 0
    for batch in test_loader:
        inputs, labels = batch.text.to(device), batch.label.to(device)
        outputs = net(inputs)
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the {} test reviews: {} %'.format(total, 100 * correct / total))

Test Accuracy of the model on the 25000 test reviews: 84.448 %
